<a href="https://colab.research.google.com/github/Chetirebochki/PythonLibruary/blob/main/12glava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#выводит на экран строку отзыва и его метку класса (positive или negative).
def pretty_print_review_and_label(i):
   print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

#Открываем файл с отзывами и записываем их построчно в список reviews, 
#используя map, чтобы удалить символ переноса строки в конце каждой строки.
g = open('reviews.txt','r') # What we know!
reviews = list(map(lambda x:x[:-1],g.readlines()))

#Закрываем файл с отзывами.
g.close()

#Открываем файл с метками классов и записываем их построчно в список labels, 
g = open('labels.txt','r') # What we WANT to know!

#используя map, чтобы перевести все буквы в верхний регистр и удалить символ переноса строки в конце каждой строки.
labels = list(map(lambda x:x[:-1].upper(),g.readlines()))

#Закрываем файл с метками классов.
g.close()


# Preprocess dataset:

import sys
# Открываем файл с отзывами в переменной f и записываем их в список raw_reviews.
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

#Открываем файл с метками классов в переменной f и записываем их в список raw_labels.
f = open('labels.txt')
raw_labels = f.readlines()
f.close()

#Разбиваем каждый отзыв на множество уникальных слов (tokens)
# и записываем результат в список tokens, используя lambda-функцию в map для удаления символов переноса строки в конце каждой строки.
tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

#Создаем пустой набор слов vocab и используем его для записи всех уникальных слов из всех отзывов в tokens.
vocab = set()

#Конвертируем vocab в список.
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

#Создаем словарь word2index, в котором каждому слову из vocab присваивается уникальный индекс.
word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

#Создаем пустой список input_dataset для записи индексов слов в каждом отзыве.
input_dataset = list()

#Проходим через каждый отзыв в списке tokens и каждое слово внутри отзыва переводим в его индекс из словаря word2index,
# если слово содержится в словаре. Индексы слов в отзыве добавляем в список sent_indices. 
#Когда все слова из отзыва переведены в индексы, добавляем список sent_indices в input_dataset.
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

#Создаем пустой список target_dataset для записи меток классов (1 - если отзыв положительный, 0 - если отзыв отрицательный).
target_dataset = list()

#Проходим через каждую метку класса в списке raw_labels и добавляем метку 1 в список target_dataset,
# если отзыв положительный (label == 'positive\n'), и метку 0, если отрицательный.
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [ ]:
import numpy as np
from collections import Counter
#собираем параметры из главы 11
alpha, iterations = (0.05, 2)
hidden_size,window,negative = (50,2,5)
weights_0_1 = (np.random.rand(len(vocab),hidden_size) - 0.5) * 0.2
weights_1_2 = np.random.rand(len(vocab),hidden_size)*0

#Умножаем матрицу весов weights01 саму на себя, 
#чтобы получить сумму квадратов весов для каждого слова из словаря. Результат записываем в массив norms.
norms = np.sum(weights_0_1 * weights_0_1,axis=1)

#Изменяем размерности массива norms до размера (количество слов в словаре, 1).
norms.resize(norms.shape[0],1)

#Нормируем веса weights01 на основе полученных квадратных норм векторов и записываем результат в массив normedweights.
normed_weights = weights_0_1 * norms

#Создаем функцию makesentvect(words), 
#которая получает на вход список слов и возвращает средний вектор взвешенных индексов слов в этом списке.
def make_sent_vect(words):
    indices = list(map(lambda x:word2index[x],filter(lambda x:x in word2index,words)))
    return np.mean(normed_weights[indices],axis=0)

#Инициализируем пустой список reviews2vectors.
reviews2vectors = list()

#Проходим через каждый отзыв в списке tokens 
#и добавляем в reviews2vectors средний вектор взвешенных индексов слов в этом отзыве, 
#построенный с помощью функции makesentvect.
for review in tokens: # tokenized reviews
    reviews2vectors.append(make_sent_vect(review))
reviews2vectors = np.array(reviews2vectors)

#Создаем функцию mostsimilarreviews(review), с помощью скалярного произведения векторов. 
#Для этого сначала получаем средний вектор взвешенных индексов слов review с помощью makesentvect, 
#затем проходим через каждый отзыв в reviews2vectors и считаем скалярное произведение его вектора со средним вектором review. 
#Результат записываем в словарь scores, где ключ - индекс отзыва, значение - скалярное произведение оценки отзыва со значением вектора review. 
#Далее из словаря scores выбираем три элемента с самыми большими значениями методом mostcommon() 
#и записываем их индексы в список mostsimilar. Затем возвращаем 40 первых символов каждого отзыва в mostsimilar.
def most_similar_reviews(review):
    v = make_sent_vect(review)
    scores = Counter()
    for i,val in enumerate(reviews2vectors.dot(v)):
        scores[i] = val
    most_similar = list()
    
    for idx,score in scores.most_common(3):
        most_similar.append(raw_reviews[idx][0:40])
    return most_similar
# Вызываем функцию mostsimilarreviews('boring','awful') для поиска наиболее похожих отзывов на слова "boring" и "awful".
most_similar_reviews(['boring','awful'])

['the characters are unlikeable and the sc',
 'adrian pasdar is excellent is this film ',
 'long  boring  blasphemous . never have i']

In [ ]:
import numpy as np
#создаем массивы 
a = np.array([1,2,3])
b = np.array([0.1,0.2,0.3])
c = np.array([-1,-0.5,0])
d = np.array([0,0,0])

#Создаем двумерный массив identity, представляющий собой единичную матрицу размера 3x3.
identity = np.eye(3)
print(identity)
print(a.dot(identity))
print(b.dot(identity))
print(c.dot(identity))
print(d.dot(identity))

this = np.array([2,4,6])
movie = np.array([10,10,10])
rocks = np.array([1,1,1])
#Выполняем поэлементное сложение массивов this, movie и rocks и выводим результат на экран.
print(this + movie + rocks)
#Выполняем матричное умножение массива this на identity, прибавляем к результату массив movie,
# выполняем матричное умножение результата на identity  
#и прибавляем массив rocks. Выводим полученный результат на экран.
print((this.dot(identity) + movie).dot(identity) + rocks)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[1. 2. 3.]
[0.1 0.2 0.3]
[-1.  -0.5  0. ]
[0. 0. 0.]
[13 15 17]
[13. 15. 17.]


In [2]:
import numpy as np
#высчитываем вероятности для каждой строки матриццы
def softmax(x_):
    x = np.atleast_2d(x_)
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

#инициализируется словарь word_vects с векторами для каждого слова.
# Каждый вектор состоит из трех значений 0.
word_vects = {}
word_vects['yankees'] = np.array([[0.,0.,0.]])
word_vects['bears'] = np.array([[0.,0.,0.]])
word_vects['braves'] = np.array([[0.,0.,0.]])
word_vects['red'] = np.array([[0.,0.,0.]])
word_vects['socks'] = np.array([[0.,0.,0.]])
word_vects['lose'] = np.array([[0.,0.,0.]])
word_vects['defeat'] = np.array([[0.,0.,0.]])
word_vects['beat'] = np.array([[0.,0.,0.]])
word_vects['tie'] = np.array([[0.,0.,0.]])

#sent2output является матрицей случайных чисел размером 3 на количество слов (здесь 9 слов)
sent2output = np.random.rand(3,len(word_vects))

#Затем определяются две матрицы identity и layer_0. 
#Layer_0 является вектором, соответствующим слову "red".
identity = np.eye(3)
layer_0 = word_vects['red']
#Затем слои layer_1 и layer_2 вычисляются путем умножения векторов на identity 
 #(единичную матрицу) и добавления векторов слов "socks" и "defeat" соответственно.
layer_1 = layer_0.dot(identity) + word_vects['socks']
layer_2 = layer_1.dot(identity) + word_vects['defeat']

#Наконец, прогнозы получаются путем прохождения layer_2 через матрицу sent2output с последующим применением функции softmax. 
#Напечатанный результат представляет собой матрицу вероятностей для каждой строки матрицы pred.
pred = softmax(layer_2.dot(sent2output))
print(pred)

[[0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
  0.11111111 0.11111111 0.11111111]]


Как нам вернуться к этому?

In [3]:
#Задаем сид для генератора псевдослучайных чисел библиотеки NumPy, 
#чтобы получить одинаковый результат каждый раз при запуске программы
y = np.array([1,0,0,0,0,0,0,0,0]) # target one-hot vector for "yankees"

## вычисляем разницу между предсказанием и целевым значением
pred_delta = pred - y
## вычисляем погрешность слоя скрытого состояния output
layer_2_delta = pred_delta.dot(sent2output.T)

defeat_delta = layer_2_delta * 1 # can ignore the "1" like prev. chapter

# вычисляем погрешность слоя скрытого состояния hidden
layer_1_delta = layer_2_delta.dot(identity.T)
socks_delta = layer_1_delta * 1 # again... can ignore the "1"

# вычисляем погрешность входного слоя input
layer_0_delta = layer_1_delta.dot(identity.T)

 # коэффициент обучения
alpha = 0.01


# обновляем вектора 
word_vects['red'] -= layer_0_delta * alpha
word_vects['socks'] -= socks_delta * alpha
word_vects['defeat'] -= defeat_delta * alpha
# обновляем матрицу весов между входным и скрытым слоями
identity -= np.outer(layer_0,layer_1_delta) * alpha
# обновляем матрицу весов между скрытым и выходным слоями
identity -= np.outer(layer_1,layer_2_delta) * alpha
# обновляем матрицу весов между скрытым слоем и выходом
sent2output -= np.outer(layer_2,pred_delta) * alpha

Начнем обучение

In [5]:
import sys,random,math
from collections import Counter
import numpy as np

f = open('qa1_single-supporting-fact_train.txt','r')
raw = f.readlines()
f.close()

tokens = list()
for line in raw[0:1000]:
  #добавление в список tokens нижнего регистра исходной строки без символов перевода строки и удалением первого слова в строке
    tokens.append(line.lower().replace("\n","").split(" ")[1:])
#вывод первых трех элементов списка tokens
print(tokens[0:3])

[['mary', 'moved', 'to', 'the', 'bathroom.'], ['john', 'went', 'to', 'the', 'hallway.'], ['where', 'is', 'mary?', '\tbathroom\t1']]


In [9]:
#создание пустого множества
vocab = set()
for sent in tokens:#цикл для каждой строки в списке tokens
    for word in sent:#цикл для каждой строки в списке tokens
        vocab.add(word)#добавление слова в множество

# перевод множества в список
vocab = list(vocab)

# перевод множества в список
word2index = {}
# циклический проход каждой пары элемента и индекса в списке vocab и соответствующему вхождению словаря word2index
for i,word in enumerate(vocab):
    word2index[word]=i
    

def words2indices(sentence):
    idx = list()
    #добавление значения для каждого слова из word2index по соответствующему индексу
    for word in sentence:
        idx.append(word2index[word])
    return idx

def softmax(x):
  #расчет экспонентных значений всех значений массива x, уменьшенных на смещение np.max(x)
    e_x = np.exp(x - np.max(x))
    #деление каждого значения в массиве на сумму всех значений в массиве ex и возврат нового массива
    return e_x / e_x.sum(axis=0)

In [10]:
#Задаем сид для генератора псевдослучайных чисел библиотеки NumPy,
# чтобы получить одинаковый результат каждый раз при запуске программы
np.random.seed(1)
#Задаем размерность вектора эмбеддинга слов.
embed_size = 10

# word embeddings
#Создаем матрицу векторов эмбеддинга слов. Используем генератор псевдослучайных чисел библиотеки NumPy,
# чтобы заполнить матрицу случайными значениями. Отнимаем 0.5, чтобы значения находились в диапазоне [-0.5, 0.5].
#  Затем умножаем на 0.1, чтобы значения находились в диапазоне [-0.05, 0.05].
embed = (np.random.rand(len(vocab),embed_size) - 0.5) * 0.1
#Создаем единичную матрицу заданной размерности, чтобы использовать её для перехода от одного состояния скрытого слоя к другому.
# embedding -> embedding (initially the identity matrix)
recurrent = np.eye(embed_size)

# sentence embedding for empty sentence
#Создаем нулевой вектор размерностью embed_size для получения начального скрытого состояния модели.
start = np.zeros(embed_size)

# embedding -> output weights
decoder = (np.random.rand(embed_size, len(vocab)) - 0.5) * 0.1
#Создаем матрицу весов для перехода от эмбеддинга слов к выходному слою.
# Используем генератор псевдослучайных чисел библиотеки NumPy, чтобы заполнить матрицу случайными значениями.
# Отнимаем 0.5, чтобы значения находились в диапазоне [-0.5, 0.5].
# Затем умножаем на 0.1, чтобы значения находились в диапазоне [-0.05, 0.05].

# one hot lookups (for loss function)
#Создаем единичную матрицу, чтобы использовать ее для one-hot encoding меток слов в функции потерь.
one_hot = np.eye(len(vocab))

Прямое распространение с произвольной длиной

In [11]:
#Функция predict(sent) выполняет прогнозы для заданного текста sent.
def predict(sent):
    #Первые три строчки создают массив слоев neural network. Затем создаются пустой словарь и словарь с ключом 'hidden' и значением "start". 
#'Hidden' - это вектор признаков, который хранит скрытые состояния, которые постепенно вычисляются на каждом шаге.
    layers = list()
    layer = {}
    layer['hidden'] = start
   # Затем словарь помещается в список layers.
   # Теперь первый элемент списка layers (0) - это входной слой с начальным скрытым состоянием.
    layers.append(layer)

#Переменная loss инициализируется нулем, она будет использоваться для расчета потерь.
    loss = 0

#Цикл for выполняет прямое распространение на каждой итерации для каждого элемента текста sent.
#Запускается также внутренний цикл for, который по очереди обрабатывает длину каждого элемента текста.
    # forward propagate
    preds = list()
 # На каждой итерации словарь создается с ключом "pred",
  # который хранит значение softmax вектора текущей скрытой нейронной сети, умноженного на матрицу decoder. 
#Softmax обычно используется для преобразования логита в значения вероятности.
# Затем нейронная сеть пытается предсказать следующий термин передавая, скрытое состояние layers-1'hidden',
# и матрицу рекурсии recurrent, и добавляя вектор, представляющий текущий термин (embedsent[target_i]).
    for target_i in range(len(sent)):

        layer = {}

        # try to predict the next term
        layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder))

#Суммирование и запись значения потерь 
        loss += -np.log(layer['pred'][sent[target_i]])

        # generate the next hidden state
        #Созданный словарь добавляется в конец списка layers.
        layer['hidden'] = layers[-1]['hidden'].dot(recurrent) + embed[sent[target_i]]
        layers.append(layer)

    return layers, loss

Обратное распространение с произвольной длиной

In [12]:
# forward
for iter in range(30000):
    alpha = 0.001#скорость обучения
    sent = words2indices(tokens[iter%len(tokens)][1:])
    layers,loss = predict(sent) #возвращаем слои и потери

    # back propagate
    #происходит обход слоев в обратном порядке
    for layer_idx in reversed(range(len(layers))):
#Для слоя определяется индекс (layeridx).
#target определяется как индекс предыдущего слова.
        layer = layers[layer_idx]
        target = sent[layer_idx-1]
#Если индекс слоя layeridx больше 0, то движение производится в обратном направлении
        if(layer_idx > 0):  # if not the first layer
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = layer['output_delta'].dot(decoder.transpose())

            # if the last layer - don't pull from a later one becasue it doesn't exist
            ## # если последний слой - не извлекайте из более позднего, потому что он не существует
            if(layer_idx == len(layers)-1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())
        else: # if the first layer
            layer['hidden_delta'] = layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())

Обновление веса с произвольной длиной

In [13]:
# forward
for iter in range(30000):
    alpha = 0.001
    sent = words2indices(tokens[iter%len(tokens)][1:])

    layers,loss = predict(sent) 

    # back propagate
    for layer_idx in reversed(range(len(layers))):#цикл, перебирающий все слои нейронной сети в обратном порядке.
        layer = layers[layer_idx]#выбор текущего слоя, через который проходит обратное распространение ошибки.
        target = sent[layer_idx-1]#выбираем текущее слово, которое передается дальше или далее для предсказания, в зависимости от порядка слоя.

        if(layer_idx > 0):
          #В данной строке кода определяется, как будет обновляться скрытый слой и его веса. Также определяется,
          # как будет обновляться вектор ошибки. Если текущий слой является последним, то вектор ошибки определяется по формуле
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = layer['output_delta'].dot(decoder.transpose())

            # if the last layer - don't pull from a 
            # later one becasue it doesn't exist
            if(layer_idx == len(layers)-1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())
        else:
            layer['hidden_delta'] = layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())

    # update weights
    start -= layers[0]['hidden_delta'] * alpha / float(len(sent))

    #В данной строчке определяется, каким будет обновление весов нейронной сети.
    # Веса обновляются в соответствие с предыдущим итерационным знанием на основе текущего улучшения. 
    for layer_idx,layer in enumerate(layers[1:]):
        
        decoder -= np.outer(layers[layer_idx]['hidden'], layer['output_delta']) * alpha / float(len(sent))
        
        embed_idx = sent[layer_idx]
        embed[embed_idx] -= layers[layer_idx]['hidden_delta'] * alpha / float(len(sent))
        recurrent -= np.outer(layers[layer_idx]['hidden'], layer['hidden_delta']) * alpha / float(len(sent))
        
    if(iter % 1000 == 0):
        print("Perplexity:" + str(np.exp(loss/len(sent))))
        #Вывод perplexity является одним из способов подсчета точности модели, которая была создана,
        # с учетом действительности факта, что чем меньше значение perplexity, тем лучше работает нейронная сеть.

Perplexity:82.02110463450072
Perplexity:81.94010350865315
Perplexity:81.81574811495292
Perplexity:81.56914786505214
Perplexity:81.04263651478377
Perplexity:79.86348992916052
Perplexity:76.91877342505654
Perplexity:66.92072862414541
Perplexity:37.94708826617198
Perplexity:22.07009288920493
Perplexity:19.143939179162775
Perplexity:18.014630948257548
Perplexity:16.762129602575182
Perplexity:14.965611373128173
Perplexity:12.31726459669751
Perplexity:9.241861975008204
Perplexity:7.293871890556201
Perplexity:6.2664142563626575
Perplexity:5.548629535550116
Perplexity:5.0863678659138385
Perplexity:4.800081270629182
Perplexity:4.596557541143939
Perplexity:4.454997636216196
Perplexity:4.3686831598929246
Perplexity:4.3183997021642355
Perplexity:4.27035537820718
Perplexity:4.208223986060514
Perplexity:4.1323599623499865
Perplexity:4.049911431563135
Perplexity:3.9710411212173917


Выполнение и анализ выходных данных

In [14]:
sent_index = 4
#Вызов функции predict с аргументом words2indices(tokens[sent_index])
# и присвоение первого элемента результата функции переменной l.
# Второй элемент присваивается временной переменной, которая не будет использоваться далее.
l,_ = predict(words2indices(tokens[sent_index]))

print(tokens[sent_index])
#В цикле для каждого элемента each_layer списка l, 
#начиная со второго и до предпоследнего, выполняются следующие действия:
#a. Создание переменной input, которой присваивается значение элемента списка tokens с индексом i текущего элемента цикла.
 #   b. Создание переменной true, которой присваивается значение следующего за элементом input элемента списка tokens с индексом i.
  #  c. Создание переменной pred, которой присваивается значение из словаря vocab, соответствующее индексу элемента с максимальным значением предсказания каждого слоя в each_layer.
  #  d. Печать значения строковой переменной, содержащей текст "Prev Input:", затем значение переменной input, в случае необходимости дополненное символами пробела до 12 знаков,
  # затем значение "True:" и переменной true, дополненное символами пробела до 15 знаков, и наконец значение "Pred:" и переменной pred.
for i,each_layer in enumerate(l[1:-1]):
    input = tokens[sent_index][i]
    true = tokens[sent_index][i+1]
    pred = vocab[each_layer['pred'].argmax()]
    print("Prev Input:" + input + (' ' * (12 - len(input))) +\
          "True:" + true + (" " * (15 - len(true))) + "Pred:" + pred)

['sandra', 'moved', 'to', 'the', 'garden.']
Prev Input:sandra      True:moved          Pred:is
Prev Input:moved       True:to             Pred:to
Prev Input:to          True:the            Pred:the
Prev Input:the         True:garden.        Pred:bedroom.
